# Code for Ice-Cube 3D CNN

- Oct 29, 2018: This code just makes plots for previously trained CNNs

In [1]:
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import glob
import pickle
import time

In [2]:
%matplotlib widget
# %matplotlib inline

Useful blog for keras conv3D: http://learnandshare645.blogspot.com/2016/06/3d-cnn-in-keras-action-recognition.html

In [3]:
# keras modules
import tensorflow.keras as keras
from tensorflow.keras import layers, models, optimizers, callbacks  # or tensorflow.keras as keras
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow.keras.models import load_model



In [4]:
print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


## Modules

In [5]:
def f_get_ydata_and_wts(data_dir,f1,f2):
    ''' Load extracted data from files. Just extracting ydata and weights
    returns : inpy,weights as arrays
    '''

    inpy=np.loadtxt(data_dir+f1)
    wts=np.loadtxt(data_dir+f2)
    
    return inpy,wts
    
def f_plot_learning(history):
    '''Plot learning curves : Accuracy and Validation'''
    fig=plt.figure()
    # Plot training & validation accuracy values
    fig.add_subplot(2,1,1)
    xlim=len(history['acc'])
    
    plt.plot(history['acc'],label='Train',marker='o')
    plt.plot(history['val_acc'],label='Validation',marker='*')
#     plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xticks(np.arange(0,xlim,2))
    
    # Plot loss values
    fig.add_subplot(2,1,2)
    plt.plot(history['loss'],label='Train',marker='o')
    plt.plot(history['val_loss'],label='Validation',marker='*')
#     plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.xticks(np.arange(0,xlim,2))

    plt.legend(loc='best')


def f_plot_roc_curve(fpr,tpr):
    '''
    Module for roc plot and printing AUC
    '''
    plt.figure()
    # plt.plot(fpr,tpr)
    plt.scatter(fpr,tpr)
    plt.semilogx(fpr, tpr)
  # Zooms
    plt.xlim([10**-7,1.0])
    plt.ylim([0,1.0])
    # y=x line for comparison
    x=np.linspace(0,1,num=500)
    plt.plot(x,x)
#     plt.xscale('log')
#     plt.xlim(1e-10,1e-5)
    plt.show()

    # AUC 
    auc_val = auc(fpr, tpr)
    print("AUC: ",auc_val)


def f_plot_fit(inpy,wts,model_dict,model_loc,verbose=True):
    '''
    Plot fit results.
    '''
    
    model_save_dir=model_loc
    model_name=model_dict['name'] # string for the model
    fname_model,fname_history='model_{0}.h5'.format(model_name),'history_{0}.pickle'.format(model_name)
    
        
    ########################
    ### Read model and history
    
    ### Check if files exist
    assert os.path.exists(model_save_dir+fname_model),"Model not saved"
    assert os.path.exists(model_save_dir+fname_history),"History not saved"
    
    model=load_model(model_save_dir+fname_model)
    with open(model_save_dir+fname_history,'rb') as f:
        history= pickle.load(f)
    
    ########################
    if verbose: model.summary()
    # Plot tested model
    f_plot_learning(history)
    
    ########################
    # Get test predictions
    
    test_file_name=model_save_dir+'y-predict_model-'+str(model_name)+'.pred'
    test_y_file_name=model_save_dir+'y-test_model-'+str(model_name)+'.test'
    test_weights_file_name=model_save_dir+'wts-test_model-'+str(model_name)+'.test'    
    print("Using test prediction from previous test",test_file_name)

    assert os.path.exists(test_file_name),"y-predictions not saved"
    y_pred=np.loadtxt(test_file_name)
    ydata=np.loadtxt(test_y_file_name)
    wts=np.loadtxt(test_weights_file_name)
    assert(test_y.shape[0]==y_pred.shape[0]),"Data %s and prediction arrays %s are not of the same size"%(test_y.shape,y_pred.shape)
    
    # Condition for the case when the prediction is a 2column array 
    if len(y_pred.shape)==2: y_pred=y_pred[:,1]
    
    fpr,tpr,threshold=roc_curve(ydata,y_pred,sample_weight=wts)
    print(fpr.shape,tpr.shape,threshold.shape)
    f_plot_roc_curve(fpr,tpr)
    
    model_dict['model'],model_dict['history']=model,history
    
    return model_dict

####################################
### Modules for viewing predicted data ###
def f_plt_hist(ypred):
    ''' Plot a histogram of predictions'''
    print(ypred.shape)
    plt.figure()
    n,bins,patches=plt.hist(ypred, density=None, bins=300)
    plt.xlim(0,1)
    plt.show()
#     plt.close()

def f_get_prediction_info(fname,plot=False):
    ''' Function that prints info on predicted data.
        For example, number of zeroes and ones, plots, etc.
        First gathers data from file
    '''
    
    ## Extract predictions
    y_pred=np.loadtxt(fname)
    # Condition for the case when the prediction is a 2 column array 
    arr=y_pred[:,1] if len(y_pred.shape)==2 else y_pred
    
    # Print info and plot
    num_total=arr.shape[0]
    num_zeros=arr[arr==0.0].shape[0]
    num_ones=arr[arr==1.0].shape[0]
    print("Pred 0's:\t%s,\tPred 1's:\t%s,Total:\t %s" %(num_zeros,num_ones,num_total))
    if plot:
        ### Plot data ###
        plt.figure()
        plt.plot(arr)
        plt.title("Prediction")
        plt.show()
        
        ### Plot histogram ###
        f_plt_hist(arr)


## Read part of test data

In [6]:
if __name__=='__main__':
    
    model_loc='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/all_models_Jan28_50epochs/'

    ###Extract data : Only extract y-data and weights for tests, which has been saved already along with the model.
    ### Note!: the test file data is the same for all models, so just pick the first one. ###
#     f1,f2='y-test_model-1.test','wts-test_model-1.test'
    f1,f2='y-test_model-15.test','wts-test_model-15.test'

    inpy,wts=f_get_ydata_and_wts(model_loc,f1,f2)
    test_y,test_wts=inpy[:],wts[:]
    

In [7]:
print(inpy.shape,wts.shape,test_y.shape,test_wts.shape)

(376302,) (376302,) (376302,) (376302,)


## View predicted data

In [8]:
# fname='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/Jan11_14_models/y-predict_model-1.pred'
fname='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/all_models_Jan28_50epochs/y-predict_model-1.pred'

f_get_prediction_info(fname,plot=True)


Pred 0's:	1064,	Pred 1's:	341,Total:	 376302


FigureCanvasNbAgg()

(376302,)


FigureCanvasNbAgg()

## Plot fits

In [9]:
dict_list=[]
# for i in range(1,16):
for i in [15]:
    model_dict={'name':str(i),'description':None,'model':None,'history':None}
    print(i,model_dict)
    model_dict=f_plot_fit(test_y,test_wts,model_dict,model_loc,verbose=False)
    dict_list.append(model_dict)

15 {'name': '15', 'description': None, 'model': None, 'history': None}


FigureCanvasNbAgg()

Using test prediction from previous test /global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/all_models_Jan28_50epochs/y-predict_model-15.pred
(2,) (2,) (2,)


FigureCanvasNbAgg()

AUC:  0.5


In [13]:
# ## Comparing different models:

# for md in dict_list:
# #     print(md)
#     hist=md['history']
# #     print(hist)
# #     print(md)
#     print('Model %s'%(md['name']))
#     for key in hist.keys():
#         print(key,hist[key])
# #     print(md[''])
        

## -----------------------------------------------

In [18]:
cols=['a','b','c']
dict1=dict.fromkeys(cols,10)

In [19]:
print(dict1)

{'a': 10, 'b': 10, 'c': 10}
